## Generate Asset Class WO Data

In [1]:
asset_class = "SIG/ATC/TRK"

In [2]:
import random
import datetime
import pandas as pd
from pandas import ExcelWriter

# Config for generating work orders
no_of_wo = 250
no_of_assets = 5
asset_instll_start_date = datetime.date(2012, 1, 1)
asset_instll_end_date = datetime.date(2015, 12, 31)

In [3]:
# Generate Asset Ids (along with some random installation date)

def generate_asset_ids(no_of_asset_ids):
    # Function to generate a list of asset ids
    res = list(range(no_of_asset_ids))
    res = ['A_'+str(item) for item in res]
    return res


def sample_random_date(start_date, end_date):
    # Function to select a random date between given start and end date
    time_between_dates = end_date - start_date
    days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange(days_between_dates)
    random_date = start_date + datetime.timedelta(days=random_number_of_days)
    return random_date


def generate_asset_instll_date(asset_id_list, asset_instll_start_date, asset_instll_end_date):
    # Function to select a random install date for all given assets 
    res = {}
    for asset_id in asset_id_list:
        res[asset_id] = sample_random_date(asset_instll_start_date, asset_instll_end_date)
    return res


asset_id_list = generate_asset_ids(no_of_assets)
print("Gen. Asset Ids:", asset_id_list)

asset_instll_date = generate_asset_instll_date(asset_id_list, asset_instll_start_date, asset_instll_end_date)
print("Gen. Asset Instll Dates", asset_instll_date)

Gen. Asset Ids: ['A_0', 'A_1', 'A_2', 'A_3', 'A_4']
Gen. Asset Instll Dates {'A_0': datetime.date(2014, 3, 9), 'A_1': datetime.date(2015, 4, 25), 'A_2': datetime.date(2014, 10, 12), 'A_3': datetime.date(2015, 10, 23), 'A_4': datetime.date(2013, 11, 1)}


In [4]:
# Generate WO Ids

def get_n_digit_range(n):
    # Function to return a range of n digit numbers
    range_start = 10**(n-1)
    range_end = (10**n) - 1
    return range(range_start, range_end) 


def generate_wo_ids(no_of_wo_ids, wo_id_len):
    # Function to generate random unique work order ids
    res = random.sample(get_n_digit_range(wo_id_len), no_of_wo_ids)
    res = ['WO_'+str(wo_id) for wo_id in res]
    return res


wo_id_list = generate_wo_ids(no_of_wo, 10)

print("Number of Work Orders:", len(wo_id_list))
print("Sample Gen. Work Order Ids:", wo_id_list[:5])

Number of Work Orders: 250
Sample Gen. Work Order Ids: ['WO_1657674993', 'WO_9103430005', 'WO_2608771298', 'WO_4705761774', 'WO_7227952696']


In [5]:
# Assign asset details to work order

def assign_asset_info_for_wo(wo_id_list, asset_id_list, asset_class = 'SIG/ATC/OBU'):
    # Function to randomly add asset info to work orders
    wo_asset_id_list = []
    for wo_id in wo_id_list:
        wo_asset_id_list.append(random.sample(asset_id_list, 1)[0])
    res = pd.DataFrame(list(zip(wo_id_list, wo_asset_id_list)), columns = ['wo_num', 'asset_num'])
    res['asset_class'] = asset_class
    return res


wo_df = assign_asset_info_for_wo(wo_id_list, asset_id_list, asset_class)

print("Work Order DF Shape:", wo_df.shape)
print("WO Distribution:")
print(wo_df['asset_num'].value_counts())
wo_df.head()

Work Order DF Shape: (250, 3)
WO Distribution:
A_2    57
A_4    54
A_0    49
A_3    45
A_1    45
Name: asset_num, dtype: int64


,wo_num,asset_num,asset_class
0,WO_1657674993,A_0,SIG/ATC/TRK
1,WO_9103430005,A_0,SIG/ATC/TRK
2,WO_2608771298,A_2,SIG/ATC/TRK
3,WO_4705761774,A_3,SIG/ATC/TRK
4,WO_7227952696,A_1,SIG/ATC/TRK


In [6]:
# Assign date to work order

def assign_date_for_wo(wo_df, asset_instll_date):
    # Function to randomly select a work order date between install date and current date
    res = wo_df.copy()
    wo_date_list = []
    for _, row in wo_df.iterrows():
        start_date = asset_instll_date[row['asset_num']]
        end_date = datetime.date.today()
        wo_date = sample_random_date(start_date, end_date)
        wo_date_list.append(wo_date)
    res['wo_date'] = wo_date_list
    return res


wo_df = assign_date_for_wo(wo_df, asset_instll_date)

print("Work Order DF Shape:", wo_df.shape)
wo_df.head()

Work Order DF Shape: (250, 4)


,wo_num,asset_num,asset_class,wo_date
0,WO_1657674993,A_0,SIG/ATC/TRK,2016-08-25
1,WO_9103430005,A_0,SIG/ATC/TRK,2021-12-24
2,WO_2608771298,A_2,SIG/ATC/TRK,2020-08-03
3,WO_4705761774,A_3,SIG/ATC/TRK,2021-06-17
4,WO_7227952696,A_1,SIG/ATC/TRK,2017-12-28


In [7]:
# Save the generated work order data

def save_xls(list_dfs, xls_path):
    # Function to save a list of dataframes as sheets of an excel
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n, index=False)
    return None


asset_instll_date_df = pd.DataFrame([(k,v) for k, v in asset_instll_date.items()], columns=['asset_num', 'instll_date'])
asset_instll_date_df['asset_class'] = asset_class
save_xls([asset_instll_date_df, wo_df], "../dataset/dummy_wo-"+asset_class.replace('/','_')+".xlsx")